In [1]:
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import *
from plotly.graph_objs.scatter.marker import Line
from plotly.subplots import make_subplots
import statsmodels
import numpy as np
import re
from bs4 import BeautifulSoup
import plotly.io as pio
import matplotlib.pyplot as plt
import requests
from pydrive.drive import GoogleDrive
from io import StringIO
import plot_settings
import yfinance as yf
import json
from plotly.validators.scatter.marker import SymbolValidator

---

# link mapping 

In [2]:
# link_map = {'AAPL':'https://drive.google.com/file/d/1pbQacz0l7oAyoDpK8CMrR-4W3ZgmOM7i/view?usp=sharing'}

In [3]:
# comp_choice = 'AAPL'

In [4]:
week = datetime(2021,9,17)

---

In [10]:
GOOD_FRIDAYS = [datetime(2022,4,15), datetime(2023,4,7), datetime(2024,3,29)]
THURS_BEFORE = [x - timedelta(days=1) for x in GOOD_FRIDAYS]

### Load the file and reformat data 

In [5]:
def pull_google_drive(url):
    file_id = url.split('/')[-2]
    dwn_url = "https://drive.google.com/uc?id=" + file_id
    df = pd.read_csv(dwn_url)
    return df

In [6]:
url = 'https://drive.google.com/file/d/1pbQacz0l7oAyoDpK8CMrR-4W3ZgmOM7i/view?usp=sharing'
dfs = pull_google_drive(url)

In [7]:
def reformat_dfs(d):
    # remove Unnamed columns
    cols_keep = [x for x in d.columns if not x.startswith('Unnamed')]
    d = d[cols_keep]
    
    # rename columns
    d.columns = [x.replace(' ','_').replace('__','_').replace('.','').replace('1','_p').lower() for x in d.columns]
    
    # remove blank rows bc were unnecessary lines in original df with section headers
    d = d[d.expiration_date.notnull()]

    # reformat date columns
    date_cols = ['expiration_date','expiration_date_p']
    for c in date_cols:
        d[c] = pd.to_datetime(d[c])
        
    return d    

In [8]:
df = reformat_dfs(dfs)

In [9]:
df

,call,last,chg,bid,ask,th_val,imp_vol,delta,gam,cvol,...,chg_p,bid_p,ask_p,th_val_p,imp_vol_p,delta_p,gam_p,cvol_p,open_int_p,expiration_date_p
1,I1721D275000,118.40,-2.85,118.35,118.70,118.325000,0.0000,1.000000,0.000000,19.0,...,0.0,0.0,0.01,0.0050,982.942,-0.000233,1.164420e-05,0.0,2620.0,2021-09-17
2,I1721D287500,121.75,0.00,117.10,117.45,117.075000,0.0000,1.000000,0.000000,0.0,...,0.0,0.0,0.01,0.0050,956.988,-0.000240,1.228200e-05,0.0,393.0,2021-09-17
3,I1721D300000,118.70,0.00,115.85,116.20,115.825000,0.0000,1.000000,0.000000,0.0,...,0.0,0.0,0.01,0.0050,932.179,-0.000246,1.294060e-05,0.0,317.0,2021-09-17
4,I1721D312500,118.75,0.00,114.60,114.95,114.575000,0.0000,1.000000,0.000000,0.0,...,0.0,0.0,0.01,0.0050,908.419,-0.000253,1.362140e-05,0.0,410.0,2021-09-17
5,I1721D325000,116.46,0.00,113.35,113.70,113.325000,0.0000,1.000000,0.000000,0.0,...,0.0,0.0,0.01,0.0050,885.623,-0.000260,1.432510e-05,0.0,244.0,2021-09-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,K1921C230000,0.00,0.00,0.03,0.06,0.045000,41.7985,0.005797,0.000647,0.0,...,0.0,84.0,84.20,84.3812,0.000,-1.000000,8.664120e-09,0.0,0.0,2021-11-19
377,K1921C235000,0.00,0.00,0.03,0.06,0.044999,43.5024,0.005599,0.000603,0.0,...,0.0,89.0,89.20,89.3808,0.000,-1.000000,1.989720e-09,0.0,0.0,2021-11-19
378,K1921C240000,0.03,0.03,0.02,0.04,0.030000,43.3333,0.003871,0.000435,800.0,...,0.0,94.0,94.15,94.3804,0.000,-1.000000,4.412210e-10,0.0,0.0,2021-11-19
379,K1921C245000,0.03,0.00,0.02,0.05,0.034999,45.5893,0.004268,0.000452,0.0,...,0.0,99.0,99.15,99.3800,0.000,-1.000000,9.473900e-11,0.0,0.0,2021-11-19


---

In [14]:
df.expiration_date.unique()

array(['2021-09-17T00:00:00.000000000', '2021-09-24T00:00:00.000000000',
       '2021-10-01T00:00:00.000000000', '2021-10-08T00:00:00.000000000',
       '2021-10-15T00:00:00.000000000', '2021-10-22T00:00:00.000000000',
       '2021-10-29T00:00:00.000000000', '2021-11-19T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [17]:
single = [datetime(2021,10,1), datetime(2022,1,3)]

df[df.expiration_date.isin(single)]['expiration_date']

148   2021-10-01
149   2021-10-01
150   2021-10-01
151   2021-10-01
152   2021-10-01
153   2021-10-01
154   2021-10-01
155   2021-10-01
156   2021-10-01
157   2021-10-01
158   2021-10-01
159   2021-10-01
160   2021-10-01
161   2021-10-01
162   2021-10-01
163   2021-10-01
164   2021-10-01
165   2021-10-01
166   2021-10-01
167   2021-10-01
168   2021-10-01
169   2021-10-01
170   2021-10-01
171   2021-10-01
172   2021-10-01
173   2021-10-01
174   2021-10-01
175   2021-10-01
176   2021-10-01
177   2021-10-01
178   2021-10-01
179   2021-10-01
180   2021-10-01
181   2021-10-01
182   2021-10-01
183   2021-10-01
184   2021-10-01
185   2021-10-01
186   2021-10-01
187   2021-10-01
188   2021-10-01
189   2021-10-01
190   2021-10-01
191   2021-10-01
Name: expiration_date, dtype: datetime64[ns]

---

In [9]:
df = df[df.expiration_date==week]

In [15]:
company = df.root_sym.unique()[0]

In [10]:
dates = [pd.to_datetime(str(x)).strftime('%b %d, %Y') for x in df.expiration_date.unique()][0]

In [11]:
dates

'Sep 17, 2021'

In [13]:
datetime.strptime(dates, '%b %d, %Y')

datetime.datetime(2021, 9, 17, 0, 0)

---

### Pull google finance info 

In [30]:
yahoo = yf.Ticker(company)

In [15]:
start = week - timedelta(days=4)

In [28]:
yahoohist = yahoo.history(start = datetime(2021,11,1), end = datetime(2021,11,2))

if len(yahoohist) == 0:
    yahoo_failed = True
    print('failed')

In [30]:
yahoohist = yahoo.history(start = start, end = week + timedelta(days=1))

In [31]:
yahoohist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-09-13,150.410490,151.199332,148.533225,149.332062,102404300,0,0
2021-09-14,150.130890,150.849842,146.695901,147.904129,109296300,0,0
2021-09-15,148.343489,149.222211,146.156678,148.812805,83281300,0,0
2021-09-16,148.223670,148.752896,147.005447,148.573151,68034100,0,0
2021-09-17,148.603125,148.603125,145.547572,145.847137,129868800,0,0


In [19]:
close = yahoohist['Close'][-1]
range_st = close - close*.08
range_end = close + close*.08

In [20]:
range_st, range_end

(134.17936645507814, 157.5149084472656)

---

### Set-up and create plot 

In [21]:
pltdf1 = df.filter(['strike','cvol','expiration_date','root_sym'])
pltdf1['type'] = 'call'
pltdf2 = df.filter(['strike','cvol_p','expiration_date','root_sym'])
pltdf2['type'] = 'put'
pltdf2 = pltdf2.rename(columns={'cvol_p':'cvol'})

pltdf = pd.concat([pltdf1,pltdf2])

In [22]:
latest = pltdf[(pltdf.strike>=range_st) & (pltdf.strike<=range_end)]

tru_rangest = latest.strike.min()
tru_rangeend = latest.strike.max()

latest = latest[latest.strike==latest.strike.max()]
latest = latest.reset_index(drop=True)

In [23]:
latest

,strike,cvol,expiration_date,root_sym,type
0,157.5,1811.0,2021-09-17,AAPL,call
1,157.5,1554.0,2021-09-17,AAPL,put


In [201]:
fig = px.scatter(pltdf, 
                 x='strike', 
                 y='cvol', 
                 color='type',
                 labels={'strike':'Strike Price', 'cvol':'Volume', 'type':'Type'},
                 color_discrete_sequence=[plot_settings.color_list[1],plot_settings.color_list[4]]).update_traces(mode='lines')

fig.add_vline(x=yahoohist['Open'][0], 
              line_width=2., 
              line_dash="dot", 
              line_color="#919191",
              annotation_text=f" Open: {yahoohist['Open'][0]:,.0f}", 
              annotation_position="top",
              annotation_textangle=-90,
              annotation_font_size=10.5,
              annotation_font_color="#919191")

fig.add_vline(x=yahoohist['Close'][-1], 
              line_width=2.5, 
              line_dash="dot", 
              line_color="#121212",
              annotation_text=f" Close: {yahoohist['Close'][-1]:,.0f}", 
              annotation_align='left',
              annotation_position="top",
              annotation_textangle=-90,
              annotation_font_size=10.5,
              annotation_font_color='#121212')

fig.update_layout(template=plot_settings.dockstreet_template,
                  margin=dict(t=100),
                  legend_title="",
                  title=dict(font_size=22,
                            x=0.03,
                            y=.98,
                            yref='container',
                            text=f"<b>{company}: Calls & puts for week ending {week.strftime('%b %-d, %Y')}</b>",
                            font_color="#4c4c4c",
                            xanchor='left'),
                  legend=dict(
                        font=dict(size=14)
                    ))

fig.update_xaxes(showgrid=False,
                 range=[tru_rangest-1, tru_rangeend])

for ser in fig['data']:
    ser['name'] = ser['name'].title()
    
fig.show()

NameError: name 'pltdf' is not defined

In [25]:
pltdf[pltdf.cvol==pltdf.cvol.max()]['strike'].values[0]

147.0

---

#### Daily animation 

In [39]:
an1 = df.filter(['strike','open_int','expiration_date','root_sym'])
an1['type'] = 'call'
an2 = df.filter(['strike','open_int_p','expiration_date','root_sym'])
an2['type'] = 'put'
an2 = an2.rename(columns={'open_int_p':'open_int'})

an = pd.concat([an1,an2])

In [40]:
an['expiration_date'].unique()

array(['2021-09-17T00:00:00.000000000', '2021-09-24T00:00:00.000000000',
       '2021-10-01T00:00:00.000000000', '2021-10-08T00:00:00.000000000',
       '2021-10-15T00:00:00.000000000', '2021-10-22T00:00:00.000000000',
       '2021-10-29T00:00:00.000000000', '2021-11-19T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [41]:
date_dict = {datetime(2021,11,19):datetime(2021,11,19),
             datetime(2021,10,29):datetime(2021,11,18),
             datetime(2021,10,22):datetime(2021,11,17),
             datetime(2021,10,15):datetime(2021,11,16),
             datetime(2021,10,8):datetime(2021,11,15)}

an = an.assign(date=lambda t: t.expiration_date.map(date_dict),
               daywk=lambda t: t['date'].dt.strftime('%a'))
an = an[an.date.notnull()]

In [51]:
y_range_st = an.open_int.min()
y_range_end = an.open_int.max() + 2

In [71]:
yahoohist = yahoo.history(start = datetime(2021,11,15), end = datetime(2021,11,20))
close = yahoohist['Close'][-1]
min_close = yahoohist['Close'].min()
max_close = yahoohist['Close'].max()
range_st = min_close - min_close*.08
range_end = max_close + max_close*.08

an = an[(an.strike>=range_st) & (an.strike<=range_end)]

tru_rangest = an.strike.min()
tru_rangeend = an.strike.max()

In [72]:
yahoohist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-11-15,150.369995,151.880005,149.429993,150.000000,59222800,0,0
2021-11-16,149.940002,151.490005,149.339996,151.000000,59256200,0,0
2021-11-17,151.000000,155.000000,150.990005,153.490005,88807000,0,0
2021-11-18,153.710007,158.669998,153.050003,157.869995,137827700,0,0
2021-11-19,157.649994,161.020004,156.529999,160.550003,117305600,0,0


In [111]:
fig = px.scatter(pltdf, 
                 x='strike', 
                 y='cvol', 
                 color='type',
                 labels={'strike':'Strike Price', 'cvol':'Volume', 'type':'Type'},
                 color_discrete_sequence=[plot_settings.color_list[1],plot_settings.color_list[4]]).update_traces(mode='lines')

fig.add_vline(x=yahoohist['Open'][0], 
              line_width=2., 
              line_dash="dot", 
              line_color="#919191",
              annotation_text=f" Open: {yahoohist['Open'][0]:,.0f}", 
              annotation_position="top",
              annotation_textangle=-90,
              annotation_font_size=10.5,
              annotation_font_color="#919191")

fig.add_vline(x=yahoohist['Close'][-1], 
              line_width=2.5, 
              line_dash="dot", 
              line_color="#121212",
              annotation_text=f" Close: {yahoohist['Close'][-1]:,.0f}", 
              annotation_align='left',
              annotation_position="top",
              annotation_textangle=-90,
              annotation_font_size=10.5,
              annotation_font_color='#121212')

fig.update_layout(template=plot_settings.dockstreet_template,
                  margin=dict(t=100),
                  legend_title="",
                  title=dict(font_size=22,
                            x=0.03,
                            y=.98,
                            yref='container',
                            text=f"<b>{company}: Calls & puts for week ending {week.strftime('%b %-d, %Y')}</b>",
                            font_color="#4c4c4c",
                            xanchor='left'),
                  legend=dict(
                        font=dict(size=14)
                    ))

fig.update_xaxes(showgrid=False,
                 range=[tru_rangest-1, tru_rangeend])

for ser in fig['data']:
    ser['name'] = ser['name'].title()
    
fig.show()

NameError: name 'pltdf' is not defined

In [73]:
yahoohist = yahoohist.reset_index()
yahoohist = yahoohist.assign(daywk=lambda t: t['Date'].dt.strftime('%a'))

In [74]:
yahoohist

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,daywk
0,2021-11-15,150.369995,151.880005,149.429993,150.000000,59222800,0,0,Mon
1,2021-11-16,149.940002,151.490005,149.339996,151.000000,59256200,0,0,Tue
2,2021-11-17,151.000000,155.000000,150.990005,153.490005,88807000,0,0,Wed
3,2021-11-18,153.710007,158.669998,153.050003,157.869995,137827700,0,0,Thu
4,2021-11-19,157.649994,161.020004,156.529999,160.550003,117305600,0,0,Fri


In [202]:
days = ['Mon','Tue','Wed','Thu','Fri']

contracts = ['call','put']

fig_dict = {
    "data":[],
    "layout":{},
    "frames":[]
}

fig_dict["layout"]["xaxis"] = {"showgrid":False, 
                               "range":[tru_rangest-1, tru_rangeend+1],
                               "title":"Strike Price"}
fig_dict["layout"]["yaxis"] = {"range":[y_range_st, y_range_end],
                               "title":"Open Interest"}
fig_dict["layout"]["hovermode"] = "x"
fig_dict['layout']['template'] = plot_settings.dockstreet_template
fig_dict['layout']['margin'] = {'t':110}
fig_dict['layout']['legend'] = {'title':'',
                                'font_size':14}
fig_dict['layout']['title'] = {'font_size':22,
                               'x':0.03,
                               'y':0.98,
                               'yref':'container',
                               'text':f"<b>Calls & puts for week ending something</b>",
                               'font_color':'#4c4c4c',
                               'xanchor':'left'}

fig_dict["layout"]["updatemenus"] = [
    {'buttons': [{'args': [None, {'frame': {'duration':500, 'redraw':False},
                                'fromcurrent':True, 'transition': {'duration':300,
                                                                   'easing':'quadratic-in-out'}}],
                    'label':'Play',
                    'method':'animate'},
                 {'args': [[None], {'frame': {'duration':0, 'redraw':False},
                                      'mode': 'immediate',
                                      'transition': {'duration':0}}],
                'label':'Pause',
                'method':'animate'}],
        'direction':'left',
        'pad': {'r':10, 't':87},
        'showactive':False,
        'type':'buttons',
        'x':0.1,
        'xanchor':'right',
        'y':0,
        'yanchor':'top'
    }]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 16},
        "prefix": "Day:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make data
day = 'Mon'
for contract in contracts:
    dataset_by_day = an[an["daywk"] == day]
    dataset_by_day_and_cont = dataset_by_day[
        dataset_by_day["type"] == contract]

    data_dict = {
        "x": list(dataset_by_day_and_cont["strike"]),
        "y": list(dataset_by_day_and_cont["open_int"]),
        "mode": "lines",
        "name": contract.title()
    }
    fig_dict["data"].append(data_dict)
    
    yahoo_day = yahoohist[yahoohist.daywk==day]
    shape_dict = [go.layout.Shape(type='line',
                                  line_width=2.,
                                  line_dash='dot',
                                  line_color='#919191',
                                  x0=yahoo_day['Open'].values.tolist()[0],
                                  x1=yahoo_day['Open'].values.tolist()[0],
                                  y0=y_range_st,
                                  y1=y_range_end,
                                  xref='x',
                                  yref='y'
                                 ),
                 go.layout.Shape(type='line',
                                 line_width=2.5,
                                 line_dash='dot',
                                 line_color='#121212',
                                 x0=yahoo_day['Close'].values.tolist()[0],
                                 x1=yahoo_day['Close'].values.tolist()[0],
                                 y0=y_range_st,
                                 y1=y_range_end,
                                 xref='x',
                                 yref='y')]
    
    fig_dict['layout']['shapes'] = shape_dict
    
# make frames
for day in days:
    frame = {"data": [], "name": str(day), "layout":{}}
    for contract in contracts:
        dataset_by_day = an[an["daywk"] == day]
        dataset_by_day_and_cont = dataset_by_day[
            dataset_by_day["type"] == contract]

        data_dict = {
            "x": list(dataset_by_day_and_cont["strike"]),
            "y": list(dataset_by_day_and_cont["open_int"]),
            "mode": "lines",
            "name": contract.title()
        }
        frame["data"].append(data_dict)
    
    # add the two vertical lines
    yahoo_day = yahoohist[yahoohist.daywk==day]
    shape_dict = [go.layout.Shape(type='line',
                                  line_width=2.,
                                  line_dash='dot',
                                  line_color='#919191',
                                  x0=yahoo_day['Open'].values.tolist()[0],
                                  x1=yahoo_day['Open'].values.tolist()[0],
                                  y0=y_range_st,
                                  y1=y_range_end,
                                  xref='x',
                                  yref='y'
                                 ),
                 go.layout.Shape(type='line',
                                 line_width=2.5,
                                 line_dash='dot',
                                 line_color='#121212',
                                 x0=yahoo_day['Close'].values.tolist()[0],
                                 x1=yahoo_day['Close'].values.tolist()[0],
                                 y0=y_range_st,
                                 y1=y_range_end,
                                 xref='x',
                                 yref='y')]

    frame['layout']['shapes'] = shape_dict
#     frame['layout'] = {'shapes':shape_dict}
    
    # add annotations for each vertical line
    annotation_dict = [go.layout.Annotation(text=f" Open: {yahoo_day['Open'].values.tolist()[0]:,.0f}",
                                            font_size=10.5,
                                            font_color="#919191",
                                            showarrow=False,
                                            x=yahoo_day['Open'].values.tolist()[0],
                                            y=y_range_end,
                                            textangle=-90,
                                            yanchor='bottom'),
                       go.layout.Annotation(text=f" Close: {yahoo_day['Close'].values.tolist()[0]:,.0f}",
                                            font_size=10.5,
                                            font_color="#121212",
                                            showarrow=False,
                                            x=yahoo_day['Close'].values.tolist()[0],
                                            y=y_range_end,
                                            textangle=-90,
                                            yanchor='bottom')]
    
#     frame['layout'] = {'annotations':annotation_dict}
    frame['layout']['annotations'] = annotation_dict

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [day],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": day,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()

In [209]:
days = ['Mon','Tue','Wed','Thu','Fri']

contracts = ['call','put']

fig_dict = {
    "data":[],
    "layout":{},
    "frames":[]
}

fig_dict["layout"]["xaxis"] = {"showgrid":False, 
                               "range":[tru_rangest-1, tru_rangeend+1],
                               "title":"Strike Price"}
fig_dict["layout"]["yaxis"] = {"range":[y_range_st, y_range_end],
                               "title":"Open Interest"}
fig_dict["layout"]["hovermode"] = "x"
fig_dict['layout']['template'] = plot_settings.dockstreet_template
fig_dict['layout']['margin'] = {'t':110}
fig_dict['layout']['legend'] = {'title':'',
                                'font_size':14}
fig_dict['layout']['title'] = {'font_size':22,
                               'x':0.03,
                               'y':0.98,
                               'yref':'container',
                               'text':f"<b>Calls & puts for week ending something</b>",
                               'font_color':'#4c4c4c',
                               'xanchor':'left'}

fig_dict["layout"]["updatemenus"] = [
    {'buttons': [{'args': [None, {'frame': {'duration':500, 'redraw':False},
                                'fromcurrent':True, 'transition': {'duration':300,
                                                                   'easing':'quadratic-in-out'}}],
                    'label':'Play',
                    'method':'animate'},
                 {'args': [[None], {'frame': {'duration':0, 'redraw':False},
                                      'mode': 'immediate',
                                      'transition': {'duration':0}}],
                'label':'Pause',
                'method':'animate'}],
        'direction':'left',
        'pad': {'r':10, 't':87},
        'showactive':False,
        'type':'buttons',
        'x':0.1,
        'xanchor':'right',
        'y':0,
        'yanchor':'top'
    }]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 16},
        "prefix": "Day:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

# make data
day = 'Mon'
data_dict = []
for contract in contracts:
    dataset_by_day = an[an["daywk"] == day]
    dataset_by_day_and_cont = dataset_by_day[
        dataset_by_day["type"] == contract]

#     data_dict = {
#         "x": list(dataset_by_day_and_cont["strike"]),
#         "y": list(dataset_by_day_and_cont["open_int"]),
#         "mode": "lines",
#         "name": contract.title()
#     }
#     fig_dict["data"].append(data_dict)
    
    data_dict.append(go.Scatter(x=dataset_by_day_and_cont['strike'], 
                                y=dataset_by_day_and_cont['open_int'], 
                                mode='lines',
                                name=contract.title()))
    
fig_dict['data'] = data_dict
    
yahoo_day = yahoohist[yahoohist.daywk==day]
shape_dict = [go.layout.Shape(type='line',
                              line_width=2.,
                              line_dash='dot',
                              line_color='#919191',
                              x0=yahoo_day['Open'].values.tolist()[0],
                              x1=yahoo_day['Open'].values.tolist()[0],
                              y0=y_range_st,
                              y1=y_range_end,
                              xref='x',
                              yref='y'
                             ),
             go.layout.Shape(type='line',
                             line_width=2.5,
                             line_dash='dot',
                             line_color='#121212',
                             x0=yahoo_day['Close'].values.tolist()[0],
                             x1=yahoo_day['Close'].values.tolist()[0],
                             y0=y_range_st,
                             y1=y_range_end,
                             xref='x',
                             yref='y')]

fig_dict['layout']['shapes'] = shape_dict
    
# make frames
for day in days:
    frame = {"data": [], "name": str(day), "layout":{}}
    for contract in contracts:
        dataset_by_day = an[an["daywk"] == day]
        dataset_by_day_and_cont = dataset_by_day[
            dataset_by_day["type"] == contract]

        data_dict = {
            "x": list(dataset_by_day_and_cont["strike"]),
            "y": list(dataset_by_day_and_cont["open_int"]),
            "mode": "lines",
            "name": contract.title()
        }
        frame["data"].append(data_dict)
    
    # add the two vertical lines
    yahoo_day = yahoohist[yahoohist.daywk==day]
    shape_dict = [go.layout.Shape(type='line',
                                  line_width=2.,
                                  line_dash='dot',
                                  line_color='#919191',
                                  x0=yahoo_day['Open'].values.tolist()[0],
                                  x1=yahoo_day['Open'].values.tolist()[0],
                                  y0=y_range_st,
                                  y1=y_range_end,
                                  xref='x',
                                  yref='y'
                                 ),
                 go.layout.Shape(type='line',
                                 line_width=2.5,
                                 line_dash='dot',
                                 line_color='#121212',
                                 x0=yahoo_day['Close'].values.tolist()[0],
                                 x1=yahoo_day['Close'].values.tolist()[0],
                                 y0=y_range_st,
                                 y1=y_range_end,
                                 xref='x',
                                 yref='y')]

    frame['layout']['shapes'] = shape_dict
#     frame['layout'] = {'shapes':shape_dict}
    
    # add annotations for each vertical line
    annotation_dict = [go.layout.Annotation(text=f" Open: {yahoo_day['Open'].values.tolist()[0]:,.0f}",
                                            font_size=10.5,
                                            font_color="#919191",
                                            showarrow=False,
                                            x=yahoo_day['Open'].values.tolist()[0],
                                            y=y_range_end,
                                            textangle=-90,
                                            yanchor='bottom'),
                       go.layout.Annotation(text=f" Close: {yahoo_day['Close'].values.tolist()[0]:,.0f}",
                                            font_size=10.5,
                                            font_color="#121212",
                                            showarrow=False,
                                            x=yahoo_day['Close'].values.tolist()[0],
                                            y=y_range_end,
                                            textangle=-90,
                                            yanchor='bottom')]
    
#     frame['layout'] = {'annotations':annotation_dict}
    frame['layout']['annotations'] = annotation_dict

    fig_dict["frames"].append(frame)
    slider_step = {"args": [
        [day],
        {"frame": {"duration": 300, "redraw": False},
         "mode": "immediate",
         "transition": {"duration": 300}}
    ],
        "label": day,
        "method": "animate"}
    sliders_dict["steps"].append(slider_step)


fig_dict["layout"]["sliders"] = [sliders_dict]

fig = go.Figure(fig_dict)

fig.show()

In [210]:
fig_dict['data']

[Scatter({
     'mode': 'lines',
     'name': 'Call',
     'x': array([138., 139., 140., 141., 142., 143., 144., 145., 146., 147., 148., 149.,
                 150., 155., 160., 165., 170.]),
     'y': array([  34.,   70.,  434.,  911.,  539.,   45.,  167.,  592.,  350.,  486.,
                 1403., 1580., 4721., 7574., 9631., 4069., 2816.])
 }),
 Scatter({
     'mode': 'lines',
     'name': 'Put',
     'x': array([138., 139., 140., 141., 142., 143., 144., 145., 146., 147., 148., 149.,
                 150., 155., 160., 165., 170.]),
     'y': array([  795.,  1048.,  2734.,   611.,   929.,   724.,   846.,  1785.,  1375.,
                  3023.,  1721., 12104.,  3718.,  1296.,   581.,   295.,   123.])
 })]

--- 

---

In [222]:
dailys = []
for d in an['date'].unique():
    tmp = an[an['date']==d]
    tmp['maxvol'] = tmp.groupby('type')['open_int'].transform('max')
    dailys.append(tmp[tmp.maxvol==tmp.open_int])
    
dailys = pd.concat(dailys)

<ipython-input-222-12008081584d>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-222-12008081584d>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-222-12008081584d>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-222-120

In [223]:
dailys

,strike,open_int,expiration_date,root_sym,type,date,daywk,maxvol
216,160.0,9631.0,2021-10-08,AAPL,call,2021-11-15,Mon,9631.0
213,149.0,12104.0,2021-10-08,AAPL,put,2021-11-15,Mon,12104.0
248,150.0,122485.0,2021-10-15,AAPL,call,2021-11-16,Tue,122485.0
248,150.0,100530.0,2021-10-15,AAPL,put,2021-11-16,Tue,100530.0
293,150.0,30804.0,2021-10-22,AAPL,call,2021-11-17,Wed,30804.0
293,150.0,6379.0,2021-10-22,AAPL,put,2021-11-17,Wed,6379.0
326,150.0,3536.0,2021-10-29,AAPL,call,2021-11-18,Thu,3536.0
325,149.0,1157.0,2021-10-29,AAPL,put,2021-11-18,Thu,1157.0
360,150.0,67588.0,2021-11-19,AAPL,call,2021-11-19,Fri,67588.0
358,140.0,20809.0,2021-11-19,AAPL,put,2021-11-19,Fri,20809.0


In [224]:
yahoohist

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,daywk
0,2021-11-15,150.369995,151.880005,149.429993,150.000000,59222800,0,0,Mon
1,2021-11-16,149.940002,151.490005,149.339996,151.000000,59256200,0,0,Tue
2,2021-11-17,151.000000,155.000000,150.990005,153.490005,88807000,0,0,Wed
3,2021-11-18,153.710007,158.669998,153.050003,157.869995,137827700,0,0,Thu
4,2021-11-19,157.649994,161.020004,156.529999,160.550003,117305600,0,0,Fri


In [254]:
fg = px.scatter(dailys,
                x='daywk',
                y='strike',
                color='type',
                size='open_int',
                opacity=0.7).update_traces(mode='markers+lines')

fg.add_trace(go.Scatter(x=yahoohist.daywk,
                        y=yahoohist.Close,
                        name='Close Price',
                        marker = {'symbol':['x']*len(yahoohist),
                                 'size':10}))

fg.update_layout(template=plot_settings.dockstreet_template)

fg.show()

In [240]:
['x']*len(yahoohist)

['x', 'x', 'x', 'x', 'x']

In [255]:
yahoohist = yahoo.history(start = datetime(2022,1,14), end = datetime(2022,1,17))

In [256]:
yahoohist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-01-14,171.339996,173.779999,171.259995,172.395004,41648786,0,0
